<a href="https://colab.research.google.com/github/SergiuTULUC/Projekt-Klasifikace/blob/main/Skalovani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**tučný text**# Instalace potřebných verzí knihoven
variable_name = "" # @param {"type":"string"}
!wget 'https://drive.google.com/uc?export=download&id=1oSFOP0j25OZAuhD8YXxyQXNTdr2lUdtn' -O requirements.txt
!pip install -r requirements.txt

# Škálování dat. (pujčeno z youtubu machine lerrrning)

## Proč je potřeba škálování

Natrénujeme model pro regresi ze `sklearn` na datasetu o nemovitostech.

```python
from sklearn.datasets import fetch_california_housing
import pandas as pd

data = fetch_california_housing()
X = pd.DataFrame(data['data'], columns=data['feature_names'])
y = data['target']

X.head()
```

Rozdělíme dataset na trénovací a testovací sadu pomocí `train_test_split`.

```python
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2)
X_train.shape, X_test.shape
```

Inicializujeme model `KNeighborsRegressor`.

```python
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
```

Vyhodnotíme kvalitu modelu.

```python
from sklearn.metrics import r2_score

pred_train = knn.predict(X_train)
pred_test = knn.predict(X_test)

print(f'Train R2 {r2_score(y_train, pred_train):.2f}')
print(f'Test R2 {r2_score(y_test, pred_test):.2f}')
```

Metriky nejsou příliš dobré. Zkusíme jiný model.

```python
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
```

Opět vyhodnotíme model.

```python
pred_train = lr.predict(X_train)
pred_test = lr.predict(X_test)

print(f'Train R2 {r2_score(y_train, pred_train):.2f}')
print(f'Test R2 {r2_score(y_test, pred_test):.2f}')
```

Výsledky jsou lepší, ale stále nejsou ideální. Vyzkoušíme rozhodovací strom.

```python
from sklearn.tree import DecisionTreeRegressor

tree_1 = DecisionTreeRegressor(random_state=1, max_depth=9)
tree_1.fit(X_train, y_train)
```

Vyhodnotíme model.

```python
pred_train = tree_1.predict(X_train)
pred_test = tree_1.predict(X_test)

print(f'Train R2 {r2_score(y_train, pred_train):.2f}')
print(f'Test R2 {r2_score(y_test, pred_test):.2f}')
```

Rozhodovací strom si vede mnohem lépe. Problém tedy nebyl v datech, ale ve zvolených modelech.

### Co může být špatně s modely?

Modely jako KNN nebo lineární regrese jsou citlivé na měřítka vstupních dat. Pokud se hodnoty v různých atributech liší řádově, může to negativně ovlivnit model.

Proto je nutné data škálovat.

## Škálování dat

Existují dva hlavní způsoby škálování: **normalizace** a **standardizace**.

### Normalizace

Normalizace převede hodnoty tak, že minimum bude 0 a maximum 1:

$$x = \frac{x - min}{max - min}$$

Použijeme `MinMaxScaler`.

```python
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
mms.fit(X_train)
X_train_norm = pd.DataFrame(mms.transform(X_train), columns=X_train.columns)
X_test_norm = pd.DataFrame(mms.transform(X_test), columns=X_train.columns)
```

Opět natrénujeme modely na normalizovaných datech.

```python
knn = KNeighborsRegressor()
knn.fit(X_train_norm, y_train)
```

Vyhodnotíme výsledky.

```python
pred_train = knn.predict(X_train_norm)
pred_test = knn.predict(X_test_norm)

print(f'Train R2 {r2_score(y_train, pred_train):.2f}')
print(f'Test R2 {r2_score(y_test, pred_test):.2f}')
```

Výsledky jsou výrazně lepší!

### Standardizace

Standardizace převede hodnoty tak, že průměr bude 0 a směrodatná odchylka 1:

$$x = \frac{x - mean}{std}$$

Použijeme `StandardScaler`.

```python
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_std = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)
```

Natrénujeme modely na standardizovaných datech.

```python
knn = KNeighborsRegressor()
knn.fit(X_train_std, y_train)
```

Vyhodnotíme výsledky.

```python
pred_train = knn.predict(X_train_std)
pred_test = knn.predict(X_test_std)

print(f'Train R2 {r2_score(y_train, pred_train):.2f}')
print(f'Test R2 {r2_score(y_test, pred_test):.2f}')
```

### Shrnutí

- Škálování dat zlepšuje stabilitu modelů.
- Existují dva hlavní způsoby škálování:
  1. **Normalizace** (min = 0, max = 1) – vhodná pro modely založené na vzdálenostech (KNN).
  2. **Standardizace** (mean = 0, std = 1) – vhodná pro lineární modely (LinearRegression, SVM).
- Pro modely založené na rozhodovacích stromech (DecisionTree, RandomForest, Boosting) není škálování nutné.

